In [ ]:
# pip install --upgrade --quiet pypdf chromadb pandas_gbq google bigquery google-search-results matplotlib wikipedia numexpr langchain langsmith  langchain-core langchain-community langchain-openai 

## Setting Things Up

In [ ]:
import os
os.getenv('./.env')

openAiApiKey = os.getenv('OPENAI_API_KEY')
bigqueryProject = os.getenv('BIG_QUERY_PROJECT')
bigqueryKey = os.getenv('BIG_QUERY_KEY')

bigqueryKey

In [ ]:
from google.cloud import bigquery
from google.oauth2 import service_account
credentials = service_account.Credentials.from_service_account_file(
    bigqueryKey,
)

bigquery_client = bigquery.Client(
    project=bigqueryProject,
    credentials=credentials
)

bigquery_schema = ''

datasets = ['TrakingProviders', 'ProdReplica']

## Transform the previous typescript code to python
def get_all_tables(dataset_id):
    try:
        # Get the dataset
        tables = bigquery_client.list_tables(dataset_id)

        # Get all tables in the dataset
        print(tables)
        # Object to store table schemas
        tableSchemas = ''

        # Iterate through each table
        for table in tables:
            tableName = table.table_id

            metadata = bigquery_client.get_table(f"{dataset_id}.{tableName}")

            metadataFormatted = [f"{field.name}: {field.field_type};" for field in metadata.schema]

            schemaString = f"""
                ##########
                TABLE {dataset_id}.{tableName} {{
                    {metadataFormatted}
                }}
            """

            tableSchemas += schemaString

        return tableSchemas
    except Exception as error:
        print('Error fetching BigQuery table schemas:', error)
        raise error

for dataset in datasets:
    dump = get_all_tables(dataset)
    bigquery_schema += "########## " + dump
bigquery_schema

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from langchain_core.runnables import RunnablePassthrough
from langchain_core.tools import tool
from langchain_core.output_parsers import JsonOutputParser
from langchain.agents import AgentExecutor, load_tools, ZeroShotAgent, initialize_agent
from langchain.memory import ConversationBufferMemory

prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")
model = ChatOpenAI(
    model="gpt-3.5-turbo", 
    api_key=openAiApiKey,
    temperature=0
  )

## Indexing

### Extracting Susep Documents 

In [ ]:
import requests
from lxml import html
import PyPDF2

## scrap the following website and find the element with the xpath //*[@id="content-core"]/ul


susep_documents = ""
data = [
    {
        "url": "https://www.gov.br/susep/pt-br/assuntos/meu-futuro-seguro/seguros-previdencia-e-capitalizacao/seguros/seguro-de-automoveis",
        "xpath": "/html/body/div[2]/div[1]/main/div[2]/div"
    },
    # {
    #     "url": "https://www.gov.br/susep/pt-br/planos-e-produtos/seguros/seguro-de-automoveis",
    #     "xpath": "/html/body/div[2]/div[1]/main/div[2]/div/div[4]/ul"
    # },
    {
        "url": 'https://www.gov.br/susep/pt-br/planos-e-produtos/seguros/seguro-compreensivo',
        "xpath": '/html/body/div[2]/div[1]/main/div[2]/div/div[4]/ul'
    },
    {
        "url": "https://www.gov.br/susep/pt-br/planos-e-produtos/seguros/seguro-de-responsabilidade",
        "xpath": "/html/body/div[2]/div[1]/main/div[2]/div/div[4]/ul"
    },
    {
        "url": "https://www.gov.br/susep/pt-br/planos-e-produtos/seguros/seguro-de-pessoas",
        "xpath": "/html/body/div[2]/div[1]/main/div[2]/div"
    },
    {
        "url":"https://www.gov.br/susep/pt-br/planos-e-produtos/seguros/subitens-fora-do-menu/perguntas-e-respostas-seguro-de-pessoas-com-cobertura-de-sobrevivencia",
        "xpath":"/html/body/div[2]/div[1]/main/div[2]/div/div[4]/ul"
    },
    {
        "url": "https://www.gov.br/susep/pt-br/planos-e-produtos/seguros/seguro-de-danos",
        "xpath": "/html/body/div[2]/div[1]/main/div[2]/div"
    },
]

for d in data:
    page = requests.get(d["url"])
    tree = html.fromstring(page.content)
    element = tree.xpath(d["xpath"])
    susep_documents += "#######" + element[0].text_content()




### Setting Susep Vector Store

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, 
    chunk_overlap=50, 
    separators=["#######", "\n", "\r\n", "\r"],
)

texts = text_splitter.split_text(susep_documents)
documents = text_splitter.create_documents(texts)
print(len(documents))
db = Chroma.from_documents(documents, OpenAIEmbeddings(
    api_key=openAiApiKey,
))

susep_retriever = db.as_retriever()





### Setting Brick CG Vector Store

In [ ]:
from langchain_community.document_loaders import PyPDFLoader


text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, 
    chunk_overlap=0,
    separators=["Página"],
)

pdf_file = open("./brick_bs_cg.pdf", "rb")
pdf_reader = PyPDF2.PdfReader(pdf_file)
pdf_text = ""
for page in pdf_reader.pages:
    pdf_text += page.extract_text()
pdf_file.close()



texts = text_splitter.split_text(pdf_text)
# print(texts)
documents = text_splitter.create_documents(texts)
# print(documents)

brick_cg_db = Chroma.from_documents(documents, OpenAIEmbeddings(
    # model="gpt-3.5-turbo", 
    api_key=openAiApiKey,
))

brick_cg_retriever = brick_cg_db.as_retriever(
    search_kwargs={"k": 5, "lambda_mult": 0.25},
    search_type="mmr",
)

brick_cg_retriever.get_relevant_documents("A apólice cobre danos materiais a terceiros?")


## File Readers

In [ ]:
import csv
import pandas as pd
import json

@tool
def read_and_stringify_csv_file(file_path): 
    """This function reads a csv file and returns a stringified version of the data"""
    try:
        with open(file_path, 'r') as file:
            csv_reader = csv.reader(file)
            data = list(csv_reader)
            print(data)
        return {"file_path": file_path, "data": str(data[0:100]),}
    except Exception as error:
        return {"error": str(error)}

@tool
def read_and_stringify_xlsx_file(file_path):
    """This function reads an xlsx file and returns a stringified version of the data"""
    try:
        data = pd.read_excel(file_path)
        return {"file_path": file_path, "data": str(data[0:100])}
    except Exception as error:
        return {"error": str(error)}

@tool
def read_and_stringify_txt_file(file_path):
    """This function reads a txt file and returns a stringified version of the data"""
    try:
        with open(file_path, 'r') as file:
            data = file.read()
        return {"data": str(data[0:100]),  "file_path": file_path}
    except Exception as error:
        return {"error": str(error)}


@tool
def read_and_stringify_json_file(file_path):
    """This function reads a json file and returns a stringified version of the data"""
    try:
        with open(file_path, 'r') as file:
            data = json.load(file)
        return {"file_path": file_path, "data": str(data[0:100])}
    except Exception as error:
        return {"error": str(error)}

def translate_to_portuguese(text):
    """This function translates a text to portuguese"""
    prompt = ChatPromptTemplate.from_messages(
       ("user", """
            Resume the following and translate it to portuguese: text:{text}.
            Não de introdução e conclusão.
            Apenas retorne o texto traduzido.
            """),
    )
    

    chain = (
        prompt |
        model
    )
    res = chain.invoke({"text": text})

    return res


## Query the database

In [ ]:


def query_database(query):
    df = bigquery_client.query(query).to_dataframe()

    return df

@tool
def query_database_chain(question):
    """This function queries the database with a given question"""
    try:
        sql_base_prompt = ChatPromptTemplate.from_messages([
            ("system", """
                You are a backend developer specialized in writting Google BigQuery SQL queries.
                Return only the SQL query that you should use to retrieve the data from the database.
                You should be able to generate the best SQL query to retrieve the data from the database.
                The data that you should retrieve is related to the input that you will receive
                Try to return the fewest columns as possible. Merge columns if necessary.
                Base on these database schema to generate the best SQL query:
                {context}
            """),
            ("user", """
                Generate a query for: {input}
            """)
        ])

        def format_sql(sql):
            return sql.replace('"', '').replace('\\n', ' ').replace('\\t', ' ').replace('```sql', '').replace('```', '').strip()
        
        sql_parser = StrOutputParser().pipe(format_sql)

        validation_prompt = ChatPromptTemplate.from_messages([
            ("system", """
                You are a SQL expert. 
                You are a Google BigQuery SQL master.
                You know everything about the SQL sintax of Google BigQuery.
                You are very good at verifying SQL queries and making them better.

                When you see a SQL query be carefult about the following:
                - Check date functions sintax.
                    -- Examples:   
                        --- DATE_ADD(date_expression, INTERVAL int64_expression date_part). date_part must be one off: DAY, WEEK, MONTH, QUARTER, YEAR.
                        --- DATE_DIFF(date_expression_a, date_expression_b, date_part). date_part must be one off: DAY, WEEK, MONTH, QUARTER, YEAR.
                        --- DATE_SUB(date_expression, INTERVAL int64_expression date_part). date_part must be one off: DAY, WEEK, MONTH, QUARTER, YEAR.
                        --- DATE_TRUNC(date_expression, date_part).  date_part must be one off: DAY, WEEK, MONTH, QUARTER, YEAR.
                        --- EXTRACT(date_part FROM date_expression). date_part must be one off: DAYOFWEEK, DAY, DAYOFYEAR, WEEK, MONTH, QUARTER, YEAR.
                        --- FORMAT_DATE(date_expression, format_string). format_string must be made off: %Y, %m, %d, %H, %M, %S.
                        --- PARSE_DATE(format_string, date_string). format_string must be made off: %Y, %m, %d, %H, %M, %S.
                        --- TIMESTAMP_ADD(timestamp_expression, INTERVAL int64_expression date_part). date_part must be one off: MICROSECOND, MILLISECOND, SECOND, MINUTE, HOUR, DAY, WEEK, MONTH, QUARTER, YEAR.
                        --- TIMESTAMP_DIFF(timestamp_expression_a, timestamp_expression_b, date_part). date_part must be one off: MICROSECOND, MILLISECOND, SECOND, MINUTE, HOUR, DAY, WEEK, MONTH, QUARTER, YEAR.
                        --- TIMESTAMP_SUB(timestamp_expression, INTERVAL int64_expression date_part). date_part must be one off: MICROSECOND, MILLISECOND, SECOND, MINUTE, HOUR, DAY, WEEK, MONTH, QUARTER, YEAR.
                        --- TIMESTAMP_TRUNC(timestamp_expression, date_part). date_part must be one off: MICROSECOND, MILLISECOND, SECOND, MINUTE, HOUR, DAY, WEEK, MONTH, QUARTER, YEAR.
                        --- DATETIME_ADD(datetime_expression, INTERVAL int64_expression date_part). date_part must be one off: MICROSECOND, MILLISECOND, SECOND, MINUTE, HOUR, DAY, WEEK, MONTH, QUARTER, YEAR.
                        --- DATETIME_SUB(datetime_expression, INTERVAL int64_expression date_part). date_part must be one off: MICROSECOND, MILLISECOND, SECOND, MINUTE, HOUR, DAY, WEEK, MONTH, QUARTER, YEAR.
                        --- DATETIME_TRUNC(datetime_expression, date_part). date_part must be one off: MICROSECOND, MILLISECOND, SECOND, MINUTE, HOUR, DAY, WEEK, MONTH, QUARTER, YEAR.
                Return only the SQL query that should be use to retrieve the data from the database.
                Do not give introduction or conclusion. Answer with only the SQL query.
                Validate this SQL: {result}
            """)
        ])
            

        chain = (
            sql_base_prompt |
            model |
            {"result": sql_parser} |
            validation_prompt |
            model |
            sql_parser |
            query_database
        )

        res = chain.invoke({"input": question, "context": bigquery_schema}) 

        return res
    except Exception as error:
        return {"error": str(error)}
# query_database_chain("Quais placas andaram mais rápido no mês de fevereiro de 2024?")
        

## Plotting Charts

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def convert_to_json_format(text):
    prompt = ChatPromptTemplate.from_messages(
        [("system", """
            You are a backend developer specialized in converting text to JSON format.
            You are very good at converting text to JSON format.
            You are very good at identifying keys in a text imagining that it would be plot into a graph.
            You divide the data into two keys:
                - x: The x values of the graph.
                - y: The y values of the graph.
            You do not give introduction or conclusion. Answer with only the JSON format.
         """),
         ("user", """
            Convert the following text to a JSON format.
            text={text}  
            """)]
     )
    
    chain = (
        prompt |
        model |
        JsonOutputParser()
    )

    res = chain.invoke({"text": text})

    return res

@tool
def plot_line_chart(data):
    """This function receives a dictionary with x and y values and plots a line chart"""

    try:
        json = convert_to_json_format(data)


        # return {"data_type": type(json), "data": str(json)}
        x = np.array(json['x'])
        y = np.array(json['y'])
        
        # plt.xticks(np.arange(0, len(json['x']), step=10), json['x'][::10])
        # plt.yticks(np.arange(0, len(json['y']), step=10), json['y'][::10])

        plt.plot(x, y)
        plt.show()
        return {"data": "Line chart plotted"}
    except Exception as error:
        return {"error": str(error), "message": "Try to solve the problem and run the tool again. If needed, query the database for the data again."}

@tool
def plot_bar_chart(data):
    """This function receives a dictionary with x and y values and plots a bar chart"""

    try:
        json = convert_to_json_format(data)


        # return {"data_type": type(json), "data": str(json)}
        x = np.array(json['x'])
        y = np.array(json['y'])
        
        # plt.xticks(np.arange(0, len(json['x']), step=10), json['x'][::10])
        # plt.yticks(np.arange(0, len(json['y']), step=10), json['y'][::10])

        plt.bar(x, y)
        plt.show()
        return {"data": "Bar chart plotted"}
    except Exception as error:
        return {"error": str(error), "message": "Try to solve the problem and run the tool again. If needed, query the database for the data again."}

@tool
def plot_pie_chart(data):
    """This function receives a dictionary with x and y values and plots a pie chart"""

    try:
        json = convert_to_json_format(data)

        float_y = [float(i) for i in json['y']]
        # return {"data_type": type(json), "data": str(json)}
        x = np.array(json['x'])
        y = np.array(float_y)
        
        # plt.xticks(np.arange(0, len(json['x']), step=10), json['x'][::10])
        # plt.yticks(np.arange(0, len(json['y']), step=10), json['y'][::10])

        plt.pie(y, labels=x, autopct='%1.1f%%')
        plt.show()
        return {"data": "Pie chart plotted"}
    except Exception as error:
        return {"error": str(error), "message": "Try to solve the problem and run the tool again. If needed, query the database for the data again."}

# @tool
# def plot_histogram(data):
#     """This function receives a dictionary with x values and plots a histogram chart"""

#     try:
#         json = convert_to_json_format(data)

#         # return {"data_type": type(json), "data": str(json)}
#         x = np.array(json['x'])
        
#         # plt.xticks(np.arange(0, len(json['x']), step=10), json['x'][::10])
#         # plt.yticks(np.arange(0, len(json['y']), step=10), json['y'][::10])

#         plt.hist(x, bins=10)
#         plt.show()
#         return {"data": "Histogram chart plotted"}
#     except Exception as error:
#         return {"error": str(error), "message": "Try to solve the problem and run the tool again. If needed, query the database for the data again."}

## Insurance RAG

In [ ]:
from operator import itemgetter

def get_susep_relevant_documents(question):
    try:
        content = ""
        documents = susep_retriever.get_relevant_documents(question)
        for document in documents:
            content += document.page_content
        return content
    
    except Exception as error:
        return {"error": str(error)}
    
def audit(data):
    print(data)
    return data

@tool
def susep_helper(question):
    """This function queries the SUSEP documents with a given question to answer insurance related questions."""
    try:
        
        prompt = ChatPromptTemplate.from_messages(
            [
                ("system", """
                    SUSEP is the Brazilian insurance regulatory agency.
                    You are an insurance agent specialized in SUSEP documents.
                    You are very good at finding information in SUSEP documents.
                    You are very good at generating the best response to insurance related questions.
                    Answer it using the following context:
                    {context}
                 """),
                ("user", """
                Answer this question as best as you can: {question}
            """)]
        )

        chain = (
            {"context": itemgetter("question") | susep_retriever,"question": itemgetter("question")} 
            | prompt 
            | model 
        )

        res = chain.invoke({"question": question})

        return res.content
        
    except Exception as error:
        return {"error": str(error)}
susep_helper("O que é um seguro de APP?")

## Brick CG RAG

In [ ]:
from operator import itemgetter

@tool
def brick_cg_helper(question):
    """This function queries the Brick Seguros documents with a given question to answer insurance related questions related to Brick Seguros."""
    try:
        
        prompt = ChatPromptTemplate.from_messages(
            [
                ("system", """
                    Brick Seguros is a Brazilian insurance company that you work for.
                    You are an insurance agent specialized in understanding Brick Seguros insurance terms.
                    You are very good at finding information in Brick Seguros documents.
                    You are very good at generating the best response to insurance related questions.
                    {context}
                 """),
                ("user", """
                Answer this question as best as you can: {question}
            """)]
        )

        chain = (
            {"context": itemgetter("question") | brick_cg_retriever,"question": itemgetter("question")} 
            | prompt 
            | model 
        )

        res = chain.invoke({"question": question})

        return res.content
        
    except Exception as error:
        return {"error": str(error)}
# susep_helper("O que é um seguro de APP?")

## Quoting Insurance Policy

In [ ]:
file_reader_agent = initialize_agent(
    agent_name="file_reader_agent",
    tools=[read_and_stringify_csv_file, read_and_stringify_xlsx_file, read_and_stringify_txt_file, read_and_stringify_json_file],
    llm=model
)


def read_and_stringify_file(file_path):
    return file_reader_agent.run(file_path)


def generate_script_from_file(data):
    """
    This function generates a python script to transform the data into a pandas dataframe.
    It should always run before quoting vehicle data.
    """
    translation_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", """
            Você é especialista em identificar a chave de campos desejados em um dataset. 
            Você é muito bom em escrever jsons.
            Não escreva nenhuma introdução ou conclusão.
            Retorne apenas o json que mapeia os campos do dataset para as chaves desejadas.
            """),
            ("user", """
            Gere um json que mapeie os campos do dataset para as chaves desejadas.:
            
            --
            Schema: 
                - plate: $PLATE_ATTRIBUTE_KEY
                - make: $MAKE_ATTRIBUTE_KEY
                - model: $MODEL_ATTRIBUTE_KEY
                - year: $YEAR_ATTRIBUTE_KEY

            --
            Dataset recebidos: {data}
            """),
        ]
    )

    chain = (
        translation_prompt
        | model
        | JsonOutputParser()
    )

    return chain.invoke({"data": data})
    
@tool
def quote_vehicles(file_path):
    """
        This function is used to quote vehicles. It receives a file with the content of the vehicles and returns a list of quotes.
    """
    
    try: 
        file_path_ref = ""
        
        file_path_arr = file_path.split("= ")
        if(len(file_path_arr) > 1):
            file_path_ref = file_path_arr[1].replace('"', "").strip()
        else:
            file_path_ref = file_path

        file_content = read_and_stringify_file(file_path_ref)

        attr_map = generate_script_from_file(file_content)
        print(attr_map)

        file_extension = file_path_ref.split(".")[-1]

        vehicles = []
        if file_extension == "json":
            content = {}
            with open(file_path_ref, 'r') as file:
                content = json.load(file)
                
                for vehicle in content:
                    vehicles.append({
                        "plate": vehicle[attr_map["plate"]],
                        "make": vehicle[attr_map["make"]],
                        "model": vehicle[attr_map["model"]],
                        "year": vehicle[attr_map["year"]],
                    })
        if file_extension == "csv":
            content = pd.read_csv(file_path_ref)
            print(content)
            for index, row in content.iterrows():
                vehicles.append({
                    "plate": row[attr_map["plate"]],
                    "make": row[attr_map["make"]],
                    "model": row[attr_map["model"]],
                    "year": row[attr_map["year"]],
                })
            
        print(vehicles)
        
        quotes = []
        
        for vehicle in vehicles:
            price = 1000

            if int(vehicle["year"]) < 2020:
                price += 500

            quotes.append({
                "plate": vehicle["plate"],
                "make": vehicle["make"],
                "model": vehicle["model"],
                "year": vehicle["year"],
                "price": price
            })

        return quotes
    except Exception as error:
        return {"error": str(error), "message": "Make sure you are only passing the file address as a parameter."}

## Policy Issuance

In [ ]:

@tool
def issue_policy(data):
    """
    This function receives a list of quotes and issues a policy for them.
    """

    return "Policy issued for the following vehicles: " + str(data)

## Initializing the Agent

In [ ]:
loaded_tools = load_tools(["wikipedia", "serpapi", "llm-math"], from_llm=model, llm=model)
tools = [
    read_and_stringify_csv_file,
    read_and_stringify_xlsx_file,
    read_and_stringify_txt_file,
    read_and_stringify_json_file,
    plot_line_chart,
    plot_bar_chart,
    plot_pie_chart,
    issue_policy,
    susep_helper,
    brick_cg_helper,
    # plot_histogram,
    query_database_chain,
    quote_vehicles,
    loaded_tools[0],
    loaded_tools[1],
    loaded_tools[2]
]

prefix = """Have a conversation with a human answering the following questions as best you can. You have access to the following tools:"""
suffix = """Begin!"

{chat_history}
Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["input", "chat_history", "agent_scratchpad"],
)

memory = ConversationBufferMemory(memory_key="chat_history")

llm_chain = LLMChain(llm=model, prompt=prompt)

agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools, verbose=True)

agent_chain = AgentExecutor.from_agent_and_tools(
    agent=agent, 
    tools=tools, 
    verbose=True, 
    memory=memory,
    handle_parsing_errors=True
)

# log the prompt from agent


f'"Have a conversation with a human answering the following questions as best you can. You have access to the following tools:\n\nread_and_stringify_csv_file: read_and_stringify_csv_file(file_path) - This function reads a csv file and returns a stringified version of the data\nread_and_stringify_xlsx_file: read_and_stringify_xlsx_file(file_path) - This function reads an xlsx file and returns a stringified version of the data\nread_and_stringify_txt_file: read_and_stringify_txt_file(file_path) - This function reads a txt file and returns a stringified version of the data\nread_and_stringify_json_file: read_and_stringify_json_file(file_path) - This function reads a json file and returns a stringified version of the data\nplot_line_chart: plot_line_chart(data) - This function plots a line chart\nplot_bar_chart: plot_bar_chart(data) - This function plots a bar chart\nplot_pie_chart: plot_pie_chart(data) - This function plots a pie chart\nplot_histogram: plot_histogram(data) - This function plots a histogram\nquote_vehicles: quote_vehicles(file_path) - This function is used to quote vehicles. It receives a file with the content of the vehicles and returns a list of quotes.\nwikipedia: A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.\nSearch: A search engine. Useful for when you need to answer questions about current events. Input should be a search query.\nCalculator: Useful for when you need to answer questions about math.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [read_and_stringify_csv_file, read_and_stringify_xlsx_file, read_and_stringify_txt_file, read_and_stringify_json_file, plot_line_chart, plot_bar_chart, plot_pie_chart, plot_histogram, quote_vehicles, wikipedia, Search, Calculator]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!"\n\n{chat_history}\nQuestion: {input}\n{agent_scratchpad}'"

In [ ]:
class BrickAi:
    def __init__(self, agent_chain):
        self.agent_chain = agent_chain

    def run(self, input):
        content = self.agent_chain.run(input)
        res = translate_to_portuguese(str(content))
        return res.content
    
brick_ai = BrickAi(agent_chain)

## Testing

### Wikipedia and Google Search Support

In [ ]:
# brick_ai.run("O que é o teorema do limite central?")

# brick_ai.run("O que são goroutines em Go?")

### Files Support

In [ ]:
# brick_ai.run("Qual é o conteúdo de ./test_json.json")

# brick_ai.run("Qual é o conteúdo de ./plates.csv")

# brick_ai.run("Qual é o conteúdo de ./plates.json")

### Database Queries Support

In [ ]:
# brick_ai.run(input="""Quantas empresas entraram no mês de janeiro de 2024?""")

# brick_ai.run(input="""Quais as 3 placas que andaram mais rápido no mês de fevereiro de 2024?""")

### Graph Generation Support

In [ ]:

# brick_ai.run(input="""Use os dados do banco de dados e Gere um gráfico de linhas do número de empresas que entraram por mês em 2024""")

# brick_ai.run(input="""Gere um gráfico de barras da variação de odômetro dos 3 carros que mais rodaram em 2024""")

# brick_ai.run(input="""Gere um gráfico de pizza do número de empresas por categoria""")


### Specific Insurance Questions

In [ ]:
brick_ai.run(input="""A partir de quando a cobertura de APP começa a valer?""")


### Brick CG Related Questions

In [ ]:
brick_ai.run(input="""O sinistro é pode ser cobrado por kilometro??""")


### Insurance Quoting Support

In [ ]:
# brick_ai.run("Qual a diferença entre uma Apólice e um Endosso?")

# brick_ai.run("Faça uma cotação para os veículos no arquivo ./plates.json")

# brick_ai.run("Faça uma cotação para os veículos no arquivo ./plates.csv")

# brick_ai.run("Quanto deu a última cotação?")

# brick_ai.run("Quanto deu a primeira cotação?")

# brick_ai.run("Emita a apólice para essa última cotação. Depois me diga quanto ficou o total")

# brick_ai.run("Quanto deu a primeira cotação mesmo?")

# brick_ai.run("Quanto deu a cotação que você emitiu?")

# brick_ai.run("O que é cobertura de terceiros na cotação?")

# brick_ai.run("Quando pode ser que eu precise usar o pdf da minha apólice?")